## EDS232/CalCOFI Ocean chemistry prediction 2025
Author: Zoe Zhou

This study applies machine learning knowledge from EDS 232 at Bren school of environmental science and management to train models that predict dissolved inorganic carbon (DIC) levels in water samples collected by the [California Cooperative Oceanic Fisheries Investigations](https://calcofi.org) program. 

**Acknowledgements:**
This study contains materials created by Mateo Robbin for the course [EDS 232 - Machine Learning](https://maro406.github.io/eds-232-machine-learning/), which is part of the [UCSB Masters in Environmental Data Science](https://bren.ucsb.edu/masters-programs/master-environmental-data-science). The study also thanks CalCOFI and Dr. Erin Satterthwaite for providing this dataset.

### Set up

In [1]:
# Load library and import data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
#from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error
from scipy.stats import uniform, randint
from xgboost import plot_importance
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

### Explore Data

In [2]:
#test_data.describe()
test_data.info()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 485 non-null    int64  
 1   Lat_Dec            485 non-null    float64
 2   Lon_Dec            485 non-null    float64
 3   NO2uM              485 non-null    float64
 4   NO3uM              485 non-null    float64
 5   NH3uM              485 non-null    float64
 6   R_TEMP             485 non-null    float64
 7   R_Depth            485 non-null    int64  
 8   R_Sal              485 non-null    float64
 9   R_DYNHT            485 non-null    float64
 10  R_Nuts             485 non-null    float64
 11  R_Oxy_micromol.Kg  485 non-null    float64
 12  PO4uM              485 non-null    float64
 13  SiO3uM             485 non-null    float64
 14  TA1                485 non-null    float64
 15  Salinity1          485 non-null    float64
 16  Temperature_degC   485 non

In [3]:
train_data.describe()

,id,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Nuts,R_Oxy_micromol.Kg,Unnamed: 12,PO4uM,SiO3uM,TA1.x,Salinity1,Temperature_degC,DIC
count,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,0.0,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000
mean,727.500000,33.271315,-120.216359,0.062252,18.885812,0.085062,10.882772,193.451857,224.527854,0.374726,0.085062,146.507682,NaN,1.644869,29.171437,2256.054409,33.764094,10.901307,2150.468820
std,419.877958,0.891261,1.719873,0.284517,14.414059,0.190922,3.702193,347.486135,88.427864,0.365226,0.190922,92.421033,NaN,1.024450,28.628682,35.215125,0.398409,3.684964,113.163645
min,1.000000,30.417500,-124.000670,0.000000,0.000000,0.000000,1.250000,1.000000,44.900000,0.003000,0.000000,0.000000,NaN,0.170000,0.000000,2181.570000,32.840000,1.520000,1948.850000
25%,364.250000,32.654580,-121.844853,0.000000,1.877500,0.000000,8.185000,30.000000,149.475000,0.107000,0.000000,59.170572,NaN,0.490000,3.585000,2230.032500,33.417000,8.215000,2025.818652
50%,727.500000,33.420670,-120.025080,0.014000,22.600000,0.010000,9.900000,101.000000,202.000000,0.293500,0.010000,136.267250,NaN,1.820000,24.150000,2244.020000,33.746800,9.910000,2166.630000
75%,1090.750000,34.150520,-118.630000,0.050000,31.500000,0.090000,13.667500,252.000000,299.075000,0.577750,0.090000,244.636050,NaN,2.560000,45.675000,2279.175000,34.149450,13.667500,2252.657500
max,1454.000000,34.663330,-117.308600,8.190000,42.000000,2.750000,22.750000,3595.000000,485.900000,3.226000,2.750000,332.347700,NaN,4.280000,175.200000,2433.710000,34.676000,22.750000,2367.800000


### 1. Preprocessing
- Clean dataframe
- Assign target and feature variables
- Split into training and validation
- Scale features

In [4]:
# Drop unnamed:12 column and clean column name: TA1.x to TA1_x
train_data = train_data.drop(columns=['Unnamed: 12']).rename(columns={"TA1.x": "TA1"})
#train_data.info()

# Assign "DIC" to target and all other variables as features
y = train_data.DIC
X = train_data.drop('DIC', axis=1)

# Split data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 808)

# Skip this because scaling doesn't matter in trees 
#scaler = StandardScaler()
# Scale features in train and test
#X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
#X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X.columns)

In [5]:
#X_train_scaled.describe()

### 2. Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.

In [6]:
# Build XGB model with defined hyperparameters
xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    #eval_metric="rmse",
    early_stopping_rounds=50,
    random_state=808)

# fit model with training data and validation sets
xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Print best n_estimators from xgb model 
best_n_estimators = xgb.best_iteration
print(f"Best number of trees in baseline model is {best_n_estimators}")

[0]	validation_0-rmse:103.60214
[1]	validation_0-rmse:93.49141
[2]	validation_0-rmse:84.40208
[3]	validation_0-rmse:76.21379
[4]	validation_0-rmse:68.84616
[5]	validation_0-rmse:62.23295
[6]	validation_0-rmse:56.29077
[7]	validation_0-rmse:50.94969
[8]	validation_0-rmse:46.15471
[9]	validation_0-rmse:41.81564
[10]	validation_0-rmse:37.93915
[11]	validation_0-rmse:34.48719
[12]	validation_0-rmse:31.38575
[13]	validation_0-rmse:28.61356
[14]	validation_0-rmse:26.11167
[15]	validation_0-rmse:23.87081
[16]	validation_0-rmse:21.88645
[17]	validation_0-rmse:20.10900
[18]	validation_0-rmse:18.55481
[19]	validation_0-rmse:17.17612
[20]	validation_0-rmse:15.95254
[21]	validation_0-rmse:14.86749
[22]	validation_0-rmse:13.91016
[23]	validation_0-rmse:13.07607
[24]	validation_0-rmse:12.36903
[25]	validation_0-rmse:11.73823
[26]	validation_0-rmse:11.19206
[27]	validation_0-rmse:10.72090
[28]	validation_0-rmse:10.32095
[29]	validation_0-rmse:9.96269
[30]	validation_0-rmse:9.65621
[31]	validation_0-r

### 3. Tune Learning Rate

The `learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate.

In [ ]:
# Initialize a new model
xgb_lr = XGBRegressor(
    eval_metric='rmse', 
    random_state=808,
    early_stopping_rounds=50,
    n_estimators=best_n_estimators
    )

# Define hyperparameter distribution
param_dist = {"learning_rate": uniform(0.01, 0.3)}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_lr, param_dist, n_iter =10, scoring="neg_root_mean_squared_error",
    cv=3, verbose=2, random_state=808)

# Run random search
random_search.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Get search results as a DataFrame
cv_results = pd.DataFrame(random_search.cv_results_)

# Print best learning rate
best_lr = random_search.best_params_['learning_rate']
print(f"Best learning rate: {best_lr: .4f}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[0]	validation_0-rmse:81.78909
[1]	validation_0-rmse:58.33036
[2]	validation_0-rmse:41.90384
[3]	validation_0-rmse:30.64396
[4]	validation_0-rmse:22.92586
[5]	validation_0-rmse:17.66639
[6]	validation_0-rmse:14.10437
[7]	validation_0-rmse:11.76988
[8]	validation_0-rmse:10.37426
[9]	validation_0-rmse:9.52692
[10]	validation_0-rmse:8.98836
[11]	validation_0-rmse:8.69566
[12]	validation_0-rmse:8.54970
[13]	validation_0-rmse:8.46563
[14]	validation_0-rmse:8.42631
[15]	validation_0-rmse:8.37198
[16]	validation_0-rmse:8.37408
[17]	validation_0-rmse:8.38044
[18]	validation_0-rmse:8.36045
[19]	validation_0-rmse:8.32697
[20]	validation_0-rmse:8.30139
[21]	validation_0-rmse:8.30458
[22]	validation_0-rmse:8.29802
[23]	validation_0-rmse:8.30547
[24]	validation_0-rmse:8.30565
[25]	validation_0-rmse:8.29694
[26]	validation_0-rmse:8.29724
[27]	validation_0-rmse:8.28826
[28]	validation_0-rmse:8.28189
[29]	validation_0-rmse:8.27997
[30]	valid

### 4. Tune Tree-Specific Parameters

Now we have best learning rate and best number of tree, we can tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 

In [8]:
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")

Best learning rate:  0.0736
Best number of trees in baseline model is 245


In [9]:
# Initialize your model with the best_n_estinators & best_lr
xgb_tree = XGBRegressor(
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    early_stopping_rounds=50,
    random_state=808)

# Set param distribution
param_dist_tree = {
    "max_depth": randint(4, 7),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.05)
}

# Set up RandomizedSearchCV
random_search_tree = RandomizedSearchCV(
    xgb_tree, 
    param_dist_tree,
    n_iter=5,
    #scoring="accuracy",
    cv=3,
    verbose=False,
    random_state=808
    )

# Fit CV model 
random_search_tree.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-rmse:106.76613
[1]	validation_0-rmse:99.09813
[2]	validation_0-rmse:92.00951
[3]	validation_0-rmse:85.44710
[4]	validation_0-rmse:79.34479
[5]	validation_0-rmse:73.69972
[6]	validation_0-rmse:68.53199
[7]	validation_0-rmse:63.72515
[8]	validation_0-rmse:59.27894
[9]	validation_0-rmse:55.16941
[10]	validation_0-rmse:51.33947
[11]	validation_0-rmse:47.84601
[12]	validation_0-rmse:44.57809
[13]	validation_0-rmse:41.54650
[14]	validation_0-rmse:38.76628
[15]	validation_0-rmse:36.15973
[16]	validation_0-rmse:33.77510
[17]	validation_0-rmse:31.52885
[18]	validation_0-rmse:29.50119
[19]	validation_0-rmse:27.61636
[20]	validation_0-rmse:25.86480
[21]	validation_0-rmse:24.25747
[22]	validation_0-rmse:22.75839
[23]	validation_0-rmse:21.41917
[24]	validation_0-rmse:20.18626
[25]	validation_0-rmse:19.05058
[26]	validation_0-rmse:17.96898
[27]	validation_0-rmse:17.01198
[28]	validation_0-rmse:16.13285
[29]	validation_0-rmse:15.33500
[30]	validation_0-rmse:14.60235
[31]	validation_0

RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=50,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0....
                                          random_state=808, ...),
                   n_iter=5,
                   param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f1545009900>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f1545009570>,
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f1545022110>},
                   random_state=808, verbose=False)

In [10]:
# Print best tree hyperparameters
best_tree_params = random_search_tree.best_params_
print(f"Best tree parameters: {best_tree_params}")
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")

Best tree parameters: {'gamma': 0.051017335628904814, 'max_depth': 6, 'min_child_weight': 2}
Best learning rate:  0.0736
Best number of trees in baseline model is 245


### 5. Tune Stochastic Components
ow, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and 1 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and 1
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 

In [ ]:
# Initialize model for tuning stochastic components
xgb_sc = XGBRegressor(
    early_stopping_rounds=50,
    random_state=808,
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    **best_tree_params,
    )

# Define param dist within stochastic component 
param_dist_sc = {
    "subsample":uniform(0.5,0.6),
    "colsample_bytree":uniform(0.5,0.6)
}

# Initialize CV model
random_search_sc = RandomizedSearchCV(
    xgb_sc, param_dist_sc, n_iter=10, scoring="neg_root_mean_squared_error",
    cv=3, verbose=False, random_state=808
)

# Fit search
random_search_sc.fit(X_train, y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-rmse:106.80137
[1]	validation_0-rmse:99.12531
[2]	validation_0-rmse:92.07731
[3]	validation_0-rmse:85.52430
[4]	validation_0-rmse:79.45941
[5]	validation_0-rmse:73.82670
[6]	validation_0-rmse:68.58165
[7]	validation_0-rmse:63.77081
[8]	validation_0-rmse:59.33951
[9]	validation_0-rmse:55.15208
[10]	validation_0-rmse:51.33617
[11]	validation_0-rmse:47.84909
[12]	validation_0-rmse:44.59792
[13]	validation_0-rmse:41.56374
[14]	validation_0-rmse:38.76875
[15]	validation_0-rmse:36.20692
[16]	validation_0-rmse:33.82998
[17]	validation_0-rmse:31.63943
[18]	validation_0-rmse:29.57081
[19]	validation_0-rmse:27.68234
[20]	validation_0-rmse:25.94027
[21]	validation_0-rmse:24.30588
[22]	validation_0-rmse:22.87309
[23]	validation_0-rmse:21.52356
[24]	validation_0-rmse:20.28283
[25]	validation_0-rmse:19.14901
[26]	validation_0-rmse:18.11636
[27]	validation_0-rmse:17.17603
[28]	validation_0-rmse:16.29491
[29]	validation_0-rmse:15.48058
[30]	validation_0-rmse:14.75185
[31]	validation_0

In [17]:
# print best stochastic params
best_sc = random_search_sc.best_params_
print(f"Best stochastic components: {best_sc}")
print(f"Best tree parameters: {best_tree_params}")
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")

Best stochastic components: {'colsample_bytree': 0.6787441520842994, 'subsample': 0.5523627778946151}
Best tree parameters: {'gamma': 0.051017335628904814, 'max_depth': 6, 'min_child_weight': 2}
Best learning rate:  0.0736
Best number of trees in baseline model is 245


### 6. Final Model Training and Prediction
**Final Model Parameters:**

- learning_rate: 0.0736
- n_estimators: 245

Stochastic Components:
- colsample_bytree: 0.6787
- subsample: 0.5524

Tree Parameters:
- gamma: 0.0510
- max_depth: 6
- min_child_weight: 2



In [18]:
# Check test data
#test_data.info()

# Initialize final model
xgb_final = XGBRegressor(
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    **best_tree_params,
    **best_sc,
    early_stopping_rounds=50,
    eval_metric="rmse",
    random_state=808
)

# Fit model
xgb_final.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Make prediction
xgb_preds = xgb_final.predict(test_data)

[0]	validation_0-rmse:106.60890
[1]	validation_0-rmse:98.97676
[2]	validation_0-rmse:91.93199
[3]	validation_0-rmse:85.42942
[4]	validation_0-rmse:79.39399
[5]	validation_0-rmse:73.76482
[6]	validation_0-rmse:68.53392
[7]	validation_0-rmse:63.69206
[8]	validation_0-rmse:59.21831
[9]	validation_0-rmse:55.03847
[10]	validation_0-rmse:51.21632
[11]	validation_0-rmse:47.70608
[12]	validation_0-rmse:44.41031
[13]	validation_0-rmse:41.38849
[14]	validation_0-rmse:38.60197
[15]	validation_0-rmse:36.02386
[16]	validation_0-rmse:33.67153
[17]	validation_0-rmse:31.49609
[18]	validation_0-rmse:29.47897
[19]	validation_0-rmse:27.61395
[20]	validation_0-rmse:25.84908
[21]	validation_0-rmse:24.22622
[22]	validation_0-rmse:22.78378
[23]	validation_0-rmse:21.41500
[24]	validation_0-rmse:20.16873
[25]	validation_0-rmse:19.01820
[26]	validation_0-rmse:18.00229
[27]	validation_0-rmse:17.05248
[28]	validation_0-rmse:16.15913
[29]	validation_0-rmse:15.35817
[30]	validation_0-rmse:14.65806
[31]	validation_0

In [19]:
xgb_preds

array([2168.9155, 2195.2803, 2321.309 , 1992.8525, 2150.4358, 2035.3218,
       2163.2412, 2200.0059, 2274.4255, 2309.5393, 2161.4436, 2315.7812,
       2263.529 , 2216.4998, 2282.2375, 2322.4905, 2218.9014, 2270.0671,
       2315.0974, 2086.3833, 2034.6145, 2010.4702, 2026.3743, 2194.072 ,
       2314.71  , 2029.4335, 2242.3376, 2143.5183, 2239.0088, 2165.9592,
       2228.877 , 2194.031 , 2311.0085, 2006.5819, 2311.6357, 1996.8572,
       2138.2065, 2225.071 , 1990.2836, 2025.4396, 2344.9958, 1984.5474,
       2000.3392, 2264.7812, 2129.0781, 1999.8943, 2203.6338, 2289.3035,
       2007.0697, 2313.588 , 2212.0957, 2266.9495, 2128.91  , 2258.2764,
       2082.1106, 2001.0789, 2275.1206, 2318.2764, 2011.2516, 2256.5002,
       2020.4207, 2190.9946, 2025.825 , 2132.2168, 2010.1348, 2168.4639,
       2254.9868, 2311.6855, 2011.4326, 2265.9636, 2314.768 , 2005.4861,
       2134.0146, 2068.2056, 2317.7532, 2044.6282, 2024.6544, 2187.465 ,
       2210.7026, 2272.6968, 2275.0288, 2312.0605, 

In [22]:
xgb_preds_df = pd.DataFrame({"DIC": xgb_preds})
xgb_preds_df.insert(0, "id", test_data["id"].values)
xgb_preds_df.to_csv("predictions.csv", index=False)

In [23]:
xgb_preds_df.shape

(485, 2)